In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
# Store data locally for confidentiality reasons!
data_path = "/Users/victor/Documents/Confidential Dataset/ML_NOAC_NOVA_Extraction.csv"

# Read data
df2 = pd.read_csv(data_path)

KeyboardInterrupt: 

In [ ]:
df = df2.copy()

In [ ]:
# Drop columns with no email content
df = df.dropna(subset=['LastIncomingEmail__c'])

# Drop this column as it contains always the same string "Case" or "Contact"
df = df.drop(['attributes.type', 'Contact.attributes.type'], axis=1)

# drop the columns with only null values
df = df.drop(['Contact'], axis=1)

In [ ]:
#create a new column for the email content initalized to empty string
df['LastIncomingEmailContent'] = ''

In [ ]:
# convert to TYPE column to 1 if NOAC and to 0 otherwise
df['Type'] = df['Type'].apply(lambda x: 1 if x == 'NOAC' else 0)

In [ ]:
#assign the number of times ; appears in each string of LastEmailCCAddress__c in the new column cc_count
df['LastEmailCCAddressCount'] = df['LastEmailCCAddress__c'].str.count(';') + 1

#fill NaN of LastEmailCCAddress__count by 0
df['LastEmailCCAddressCount'] = df['LastEmailCCAddressCount'].fillna(0)
df['LastEmailCCAddressCount'] = df['LastEmailCCAddressCount'].astype(int)

In [ ]:
# check when SuppliedEmail has the same values as ContactEmail
test = df.apply(lambda row : 1 if row['SuppliedEmail'] == row['Contact.Email'] else 0, axis=1)

In [ ]:
# Remove the CaseNumber from the EmailTemplateSubjectDispute__c
df['EmailTemplateSubjectDispute__c'] = df.apply(lambda row : row['EmailTemplateSubjectDispute__c'].replace("Case #" + str(row['CaseNumber']), ''), axis=1)

In [ ]:
# convert Contact.attributes.url to string
df['Contact.attributes.url'] = df['Contact.attributes.url'].astype(str)

# remove /services/data/v42.0/sobjects/Contact/ from the strings in Contact.attributes.url
df['Contact.attributes.url'] = df['Contact.attributes.url'].apply(lambda x: x.replace('/services/data/v42.0/sobjects/Contact/', ''))

In [ ]:
# convert Contact.attributes.url to string
df['attributes.url'] = df['attributes.url'].astype(str)

# remove /services/data/v42.0/sobjects/Contact/ from the strings in Contact.attributes.url
df['attributes.url'] = df['attributes.url'].apply(lambda x: x.replace('/services/data/v42.0/sobjects/Case/', ''))

In [ ]:
# For the vast majority of the rows (85%), the columns SuppliedEmail and Contact.Email have the same value.
# When the values are different, one of them has NaN and the other has the email address.
# Therefore, we can fill the NaN of Contact.Email with the value of SuppliedEmail
df['Contact.Email'] = df['Contact.Email'].fillna(df['SuppliedEmail'])

# Then we can drop the column SuppliedEmail
df = df.drop(['SuppliedEmail'], axis=1)

In [ ]:
# fill NaN of LastEmailCCAddress__c by empty string
df['LastEmailCCAddress__c'] = df['LastEmailCCAddress__c'].fillna('')

In [ ]:
# Rename Topics__c in Topics
df = df.rename(columns={'Topics__c': 'Topics'})

# Rename LastIncomingEmail__c in LastIncomingEmail
df = df.rename(columns={'LastIncomingEmail__c': 'LastIncomingEmail'})

# Rename TeamName__c in TeamName
df = df.rename(columns={'TeamName__c': 'TeamName'})

# Rename RequesterEmail__c in RequesterEmail
df = df.rename(columns={'RequesterEmail__c': 'RequesterEmail'})

# Rename EmailTemplateSubjectDispute__c by EmailObject
df = df.rename(columns={'EmailTemplateSubjectDispute__c': 'EmailObject'})

# Rename LastEmailCCAddress__c by LastEmailCCAddress
df = df.rename(columns={'LastEmailCCAddress__c': 'LastEmailCCAddress'})

#Rename attributes.url by AttributesURL
df = df.rename(columns={'attributes.url': 'AttributesURL'})

#Rename Contact.attributes.url by ContactAttributesURL
df = df.rename(columns={'Contact.attributes.url': 'ContactAttributesURL'})

# Rename Contact.Email by ContactEmail
df = df.rename(columns={'Contact.Email': 'ContactEmail'})

In [ ]:
# Cleaning of the LastIncomingEmail column using BeautifulSoup
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    df.at[index, "LastIncomingEmailContent"] = BeautifulSoup(row["LastIncomingEmail"], "html").text

  0%|          | 1182/251234 [00:04<13:16, 313.81it/s]/opt/homebrew/Caskroom/miniconda/base/envs/alab_env/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
100%|██████████| 251234/251234 [17:45<00:00, 235.83it/s]


In [ ]:
# drop the column LastIncomingEmail
df = df.drop(['LastIncomingEmail'], axis=1)

In [ ]:
df.head()

,CaseNumber,Type,Topics,TeamName,RequesterEmail,EmailObject,LastEmailCCAddress,AttributesURL,ContactAttributesURL,ContactEmail,LastIncomingEmailContent,LastEmailCCAddressCount
0,100124562,0,T039,US-NOR-CMA-BKG-PREFERRED,NaN,NAM4999926A,op@easywayintl.com,5005p00002evRjHAAU,0035p00003aZAdZAAW,usa.wcportops@cma-cgm.com,\n\n\nThank you Jessica! Await your further on...,1
3,100080371,0,T036,US-BNA-CMA-CS-IMP-DIVERSIONS,NaN,MOGELIJKE COD'S F119086 // RTM,janneke.van.leeuwen@fclmarine.nl; executivedes...,5005p00002evWsuAAE,0030N00003N771RQAR,janneke.van.leeuwen@fclmarine.nl,"\nHi Nese,\n \nInmiddels de aangepaste bl’s al...",5
4,100095395,0,T035,US-NOR-CMA-BKG-PREFERRED,tina@generallogistics.net,AMEND NAM5088900 [ ref:_00D,bookings@generallogistics.net,5005p00002evX4SAAU,0030N00002x1JqPQAU,tina@generallogistics.net,"\nHello,\nPlease SPLIT subject line booking to...",1
5,100097054,0,T847,US-NOR-CMA-BKG-SPECIALTY,bookings@delongcompany.com,FW: CMA CGM - Cut-off date cha,usa.cmaexportactive@usa.cma-cgm.com,5005p00002evXIsAAM,0030N00002tXvvIQAS,bookings@delongcompany.com,"CMA CGM - Cut-off date change at - NEW YORK, N...",1
6,100097055,0,T847,US-NOR-CMA-BKG-SPECIALTY,bookings@delongcompany.com,FW: CMA CGM - Cut-off date cha,,5005p00002evXItAAM,0030N00002tXvvIQAS,bookings@delongcompany.com,"CMA CGM - Cut-off date change at - NEW YORK, N...",0


In [ ]:
# save the cleaned data 
df.to_csv('/Users/victor/Documents/Confidential Dataset/ML_NOAC_NOVA_Extraction_Cleaned.csv', index=False)

## Ideas for feature creation:

##### EmailTemplateSubjectDispute__c
1. Specific keywords: "RE" or "BookingConfirmation"

#### SuppliedEmail
1. Extract the name of the company and group the SuppliedEmail depending on the company?


1. Number of words in the email
2. Number of sentences in the email
